# Backpropagation

In [ ]:
import numpy as np
import pandas as pd
import time

Pada post-test kali ini akan membandingkan dua jenis fungsi aktivasi yang biasa digunakan dalam backpropogation

In [ ]:
#Fungsi Aktivasi Sigmoid dengan turunannya
def sig(x):
  return 1 / (1 + np.exp(-x))

def sigd(x):
  return sig(x) * (1 - sig(x))

#Fungsi Aktivasi Hyperbolic Tangent dengan turunannya
def tanh(x):
  return (np.exp(x) - np.exp(-x)) / (np.exp(x) + np.exp(-x))

def tanh(x):
  return 1 - tanh(x)**2


In [ ]:
def onehot_enc(lbl, min_val=0):
  mi = min(lbl)
  enc = np.full((len(lbl), max(lbl) - mi + 1), min_val, np.int8)

  for i, x in enumerate(lbl):
    enc[i, x - mi] = 1

  return enc

def onehot_dec(enc, mi=0):
  return [np.argmax(e) + mi for e in enc]

### a) Fungsi *Training* Backpropagation

Tulis kode ke dalam *cell* di bawah ini:

In [ ]:
def bp_fit_sig(X, target, layer_conf, max_epoch, max_error=.1, learn_rate=.1, print_per_epoch=100):
  start_time = time.time()
  np.random.seed(1)
  nin = [np.empty(i) for i in layer_conf]
  n = [np.empty(j + 1) if i < len(layer_conf) - 1 else
  np.empty(j) for i, j in enumerate(layer_conf)]
  w = np.array([np.random.rand(layer_conf[i] + 1, layer_conf[i + 1]) for i in range(len(layer_conf) - 1)])
  dw = [np.empty((layer_conf[i] + 1, layer_conf[i + 1])) for i in range(len(layer_conf) - 1)]
  d = [np.empty(s) for s in layer_conf[1:]]
  din = [np.empty(s) for s in layer_conf[1:-1]]
  epoch = 0
  mse = 1
  for i in range(0, len(n)-1):
    n[i][-1] = 1
  while (max_epoch == -1 or epoch < max_epoch) and mse > max_error:
    epoch += 1
    mse = 0
    for r in range(len(X)):
      n[0][:-1] = X[r]
      for L in range(1, len(layer_conf)):
        nin[L] = np.dot(n[L-1], w[L-1])
        n[L][:len(nin[L])] = sig(nin[L])
      e = target[r] - n[-1]
      mse += sum(e ** 2)
      d[-1] = e * sigd(nin[-1])
      dw[-1] = learn_rate * d[-1] * n[-2].reshape((-1, 1))
      for L in range(len(layer_conf) - 1, 1, -1):
        din[L-2] = np.dot(d[L-1], np.transpose(w[L-1][:-1]))
        d[L-2] = din[L-2] * np.array(sigd(nin[L-1]))
        dw[L-2] = (learn_rate * d[L-2]) * n[L-2].reshape((-1, 1))
      w += dw
    mse /= len(X)
    if print_per_epoch > -1 and epoch % print_per_epoch == 0:
      print(f'Epoch {epoch}, MSE: {mse}')
  execution = time.time() - start_time
  print("Waktu eksekusi: %s detik" % execution)
  return w, epoch, mse

In [ ]:
#Membuat fungsi training backpropagation dengan menggunakan fungsi aktivasi tanh
def bp_fit_tanh(X, target, layer_conf, max_epoch, max_error=0.1, learn_rate=0.1, print_per_epoch=100):
    input_layer, hidden_layer, output_layer = layer_conf
    w_hidden = np.random.rand(input_layer, hidden_layer)
    w_output = np.random.rand(hidden_layer, output_layer)

    for epoch in range(max_epoch):
        # Forward propagation
        hidden_input = np.dot(X, w_hidden)
        hidden_output = np.tanh(hidden_input)
        output_input = np.dot(hidden_output, w_output)
        output = np.tanh(output_input)

        # Backpropagation
        error = target - output
        mse = np.mean(error**2)
        if mse < max_error:
            print(f"Training completed after {epoch} epochs. MSE: {mse:.4f}")
            break

        # Update weights
        delta_output = error * (1 - output**2)
        delta_hidden = np.dot(delta_output, w_output.T) * (1 - hidden_output**2)
        w_output += learn_rate * np.dot(hidden_output.T, delta_output)
        w_hidden += learn_rate * np.dot(X.T, delta_hidden)

        if epoch % print_per_epoch == 0:
            print(f"Epoch {epoch}: MSE = {mse:.4f}")

    return [w_hidden, w_output, epoch]


### b) Fungsi *Testing* Backpropagation

Tulis kode ke dalam *cell* di bawah ini:

In [ ]:
def bp_predict_sig(X, w):
  n = [np.empty(len(i)) for i in w]
  nin = [np.empty(len(i[0])) for i in w]
  predict = []
  n.append(np.empty(len(w[-1][0])))
  for x in X:
    n[0][:-1] = x
    for L in range(0, len(w)):
      nin[L] = np.dot(n[L], w[L])
      n[L + 1][:len(nin[L])] = sig(nin[L])
    predict.append(n[-1].copy())
  return predict

### c) Klasifikasi dataset wine


Lakukan pelatihan pada dataset wine dengan menggunakan 2 fungsi pelatihan yang telah dibuat!

Konfigurasi kedua pelatihan harus sama (epoch, hidden layer, learning rate, dll).
Akurasi yang diharapkan di setiap pelatihan adalah > 0.98

In [ ]:
import numpy as np

In [ ]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import minmax_scale
from sklearn.metrics import accuracy_score

wine = datasets.load_wine()
X = minmax_scale(wine.data)
Y = onehot_enc(wine.target)

X_train, X_test, y_train, y_test = train_test_split(X, Y,test_size=.3,random_state=1)
#Isi jumlah layer yang digunakan dengan jumlah hidden layer #
w, ep, mse = bp_fit_sig(X_train, y_train, layer_conf=(13, 10, 3),
                        learn_rate=0.01, max_epoch=1000, max_error=0.01, print_per_epoch=25)

print(f'Epochs: {ep}, MSE: {mse}')

predict = bp_predict_sig(X_test, w)
predict = onehot_dec(predict)
y_test = onehot_dec(y_test)
accuracy = accuracy_score(predict, y_test)

print('Output:', predict)
print('True :', y_test)
print('Accuracy:', accuracy)

<ipython-input-5-6557898fc4d8>:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  w = np.array([np.random.rand(layer_conf[i] + 1, layer_conf[i + 1]) for i in range(len(layer_conf) - 1)])
<ipython-input-5-6557898fc4d8>:31: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  w += dw


Epoch 25, MSE: 0.6566042896843435
Epoch 50, MSE: 0.6552151113995929
Epoch 75, MSE: 0.6536725397232533
Epoch 100, MSE: 0.6518550921653328
Epoch 125, MSE: 0.6495851428507287
Epoch 150, MSE: 0.646555389090612
Epoch 175, MSE: 0.6421712300340066
Epoch 200, MSE: 0.6351237271673849
Epoch 225, MSE: 0.6221232351325388
Epoch 250, MSE: 0.5952074234572872
Epoch 275, MSE: 0.5478830893834353
Epoch 300, MSE: 0.4914545797614302
Epoch 325, MSE: 0.4389756405689766
Epoch 350, MSE: 0.39521458262900006
Epoch 375, MSE: 0.35570668722227045
Epoch 400, MSE: 0.3159054441726655
Epoch 425, MSE: 0.2758060557419271
Epoch 450, MSE: 0.23836409068833195
Epoch 475, MSE: 0.20585969608975932
Epoch 500, MSE: 0.17895406452669071
Epoch 525, MSE: 0.15721440065319484
Epoch 550, MSE: 0.13974750666610194
Epoch 575, MSE: 0.1256249184029705
Epoch 600, MSE: 0.11406394103318941
Epoch 625, MSE: 0.104461044866247
Epoch 650, MSE: 0.09636764584618977
Epoch 675, MSE: 0.0894537930627171
Epoch 700, MSE: 0.08347597621939318
Epoch 725, MSE:

In [ ]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import minmax_scale
from sklearn.metrics import accuracy_score

wine = datasets.load_wine()
X = minmax_scale(wine.data)
Y = onehot_enc(wine.target)

X_train, X_test, y_train, y_test = train_test_split(X, Y,test_size=.3,random_state=1)
#Isi jumlah layer yang digunakan dengan jumlah hidden layer #
result = bp_fit_tanh(X_train, y_train, layer_conf=(13, 10, 3), max_epoch=1000, max_error=0.01, learn_rate=0.004, print_per_epoch=25)

w_hidden, w_output, epochs = result

print(f'Epochs: {epochs}, MSE: {mse}')

predict = bp_predict_tanh(X_test, (w_hidden, w_output))
predict = onehot_dec(predict)
y_test = onehot_dec(y_test)
accuracy = accuracy_score(predict, y_test)

print('Output:', predict)
print('True :', y_test)
print('Accuracy:', accuracy)

Epoch 0: MSE = 0.6647
Epoch 25: MSE = 0.6596
Epoch 50: MSE = 0.5672
Epoch 75: MSE = 0.5933
Epoch 100: MSE = 0.6032
Epoch 125: MSE = 0.5324
Epoch 150: MSE = 0.5962
Epoch 175: MSE = 0.5330
Epoch 200: MSE = 0.3843
Epoch 225: MSE = 0.3672
Epoch 250: MSE = 0.2742
Epoch 275: MSE = 0.1329
Epoch 300: MSE = 0.1260
Epoch 325: MSE = 0.1034
Epoch 350: MSE = 0.1071
Epoch 375: MSE = 0.0922
Epoch 400: MSE = 0.0918
Epoch 425: MSE = 0.0823
Epoch 450: MSE = 0.0808
Epoch 475: MSE = 0.0735
Epoch 500: MSE = 0.0717
Epoch 525: MSE = 0.0661
Epoch 550: MSE = 0.0643
Epoch 575: MSE = 0.0601
Epoch 600: MSE = 0.0585
Epoch 625: MSE = 0.0553
Epoch 650: MSE = 0.0538
Epoch 675: MSE = 0.0513
Epoch 700: MSE = 0.0499
Epoch 725: MSE = 0.0479
Epoch 750: MSE = 0.0464
Epoch 775: MSE = 0.0447
Epoch 800: MSE = 0.0426
Epoch 825: MSE = 0.0424
Epoch 850: MSE = 0.0643
Epoch 875: MSE = 0.0568
Epoch 900: MSE = 0.0521
Epoch 925: MSE = 0.0496
Epoch 950: MSE = 0.0472
Epoch 975: MSE = 0.0457
Epochs: 999, MSE: 0.04622225643488231
Output:

# Pertanyaan

1.  Apa perbedaan dari penggunaan fungsi aktivasi sigmoid dengan fungsi aktivasi hyperbolic tangent?
2. Coba jelaskan alasan dari perbedaan tersebut sebisa kalian

# Jawaban

1.  Perbedaan
- Bentuk kurva

Fungsi aktivasi sigmoid memiliki bentuk seperti sigmoid, sedangkan fungsi aktivasi hyperbolic tangent memiliki bentuk seperti kurva S.

- Nilai output

Fungsi aktivasi sigmoid memiliki nilai output yang mendekati 0 dan 1, sedangkan fungsi aktivasi hyperbolic tangent memiliki nilai output yang mendekati -1 dan 1.

- Turunan

Turunan fungsi aktivasi sigmoid adalah fungsi sigmoid itu sendiri, tetapi dikalikan dengan 1 - fungsi sigmoid. Turunan fungsi aktivasi hyperbolic tangent adalah fungsi hyperbolic tangent itu sendiri.

- Kelebihan dan kekurangan

Fungsi aktivasi sigmoid lebih mudah untuk dipelajari oleh jaringan saraf tiruan. Hal ini karena fungsi sigmoid memiliki turunan yang lebih sederhana. Namun, fungsi aktivasi sigmoid dapat menyebabkan overfitting pada data yang berukuran kecil.

Fungsi aktivasi hyperbolic tangent lebih tahan terhadap overfitting. Hal ini karena fungsi aktivasi hyperbolic tangent memiliki nilai output yang lebih besar daripada fungsi aktivasi sigmoid.

2.  Alasan
- Fungsi aktivasi sigmoid

Fungsi aktivasi sigmoid memiliki bentuk kurva yang mirip dengan huruf S. Nilai output fungsi aktivasi sigmoid mendekati 0 dan 1. Turunan fungsi aktivasi sigmoid adalah fungsi sigmoid itu sendiri, tetapi dikalikan dengan 1 - fungsi sigmoid.

Fungsi aktivasi sigmoid lebih mudah untuk dipelajari oleh jaringan saraf tiruan karena memiliki turunan yang lebih sederhana. Turunan fungsi aktivasi sigmoid dapat digunakan untuk menghitung gradient, yang digunakan untuk memperbarui bobot jaringan saraf tiruan selama proses training.

Namun, fungsi aktivasi sigmoid dapat menyebabkan overfitting pada data yang berukuran kecil. Overfitting adalah kondisi di mana jaringan saraf tiruan terlalu cocok dengan data training, sehingga tidak dapat generalisasi dengan baik ke data baru.

Hal ini disebabkan karena fungsi aktivasi sigmoid memiliki nilai output yang mendekati 0 dan 1. Nilai output yang mendekati 0 dan 1 dapat menyebabkan jaringan saraf tiruan menjadi terlalu sensitif terhadap data training.

- Fungsi aktivasi hyperbolic tangent

Fungsi aktivasi hyperbolic tangent memiliki bentuk kurva yang mirip dengan kurva S. Nilai output fungsi aktivasi hyperbolic tangent mendekati -1 dan 1. Turunan fungsi aktivasi hyperbolic tangent adalah fungsi hyperbolic tangent itu sendiri.

Fungsi aktivasi hyperbolic tangent lebih tahan terhadap overfitting karena memiliki nilai output yang lebih besar daripada fungsi aktivasi sigmoid. Nilai output yang lebih besar dapat menyebabkan jaringan saraf tiruan menjadi kurang sensitif terhadap data training.

Namun, fungsi aktivasi hyperbolic tangent lebih sulit untuk dipelajari oleh jaringan saraf tiruan karena memiliki turunan yang lebih kompleks. Turunan fungsi aktivasi hyperbolic tangent dapat digunakan untuk menghitung gradient, yang digunakan untuk memperbarui bobot jaringan saraf tiruan selama proses training.